In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from pathlib import Path

DATA_DIR = Path('../input/airbus-aircrafts-sample-dataset')
img_list = list(DATA_DIR.glob('images/*.jpg'))
print("Number of images: {}".format(len(img_list)))
#print(*img_list, sep = "\n")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import PIL

pickone = random.choice(img_list)
img = PIL.Image.open(pickone)
print(img.size)
display(img)

In [ ]:
import ast

# convert a string record into a valid python object
def f(x): 
    return ast.literal_eval(x.rstrip('\r\n'))

df = pd.read_csv(DATA_DIR / "annotations.csv", 
                converters={'geometry': f})
df.head(10)

In [ ]:
def getBounds(geometry):
    try: 
        arr = np.array(geometry).T
        xmin = np.min(arr[0])
        ymin = np.min(arr[1])
        xmax = np.max(arr[0])
        ymax = np.max(arr[1])
        return (xmin, ymin, xmax, ymax)
    except:
        return np.nan

def getWidth(bounds):
    try: 
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(xmax - xmin)
    except:
        return np.nan

def getHeight(bounds):
    try: 
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(ymax - ymin)
    except:
        return np.nan

# Create bounds, width and height
df.loc[:,'bounds'] = df.loc[:,'geometry'].apply(getBounds)
df.loc[:,'width'] = df.loc[:,'bounds'].apply(getWidth)
df.loc[:,'height'] = df.loc[:,'bounds'].apply(getHeight)

# Display head
df.head(10)

In [ ]:
unique, counts = np.unique(df['class'], return_counts=True)
pd.DataFrame({'class': unique, 'count': counts})

In [ ]:
unique, counts = np.unique(df['image_id'], return_counts=True)
#per_image = np.asarray((unique, counts)).T
print("Minimum aircrafts per image: {}".format(np.min(counts)))
print("Maximum aircrafts per image: {}".format(np.max(counts)))

In [ ]:
import seaborn as sns
sns.displot(df['width'])

In [ ]:
sns.displot(df['height'])

In [ ]:
import os.path
import PIL.ImageDraw

pickone = random.choice(img_list)
img = PIL.Image.open(pickone)
draw = PIL.ImageDraw.Draw(img)

for k, row in df[df['image_id'] == os.path.basename(pickone)].iterrows():
    draw.polygon(row['geometry'], outline=(255,0,0))
    draw.text(row['geometry'][0], row['class'], fill=(255,0,0))

display(img)
#img.save('/kaggle/working/' + os.path.basename(pickone))
